In [31]:
#HR bolje
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [32]:
df = pd.read_csv("data//WA_Fn-UseC_-HR-Employee-Attrition.csv")
to_num = {"No" : 0, "Yes" : 1}
df["Attrition"] = df["Attrition"].map(to_num)
df.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,1,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,0,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,1,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,0,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,0,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


In [33]:
df.describe()
numerical_col = df.describe().columns
col_drop = ['EmployeeCount', 'EmployeeNumber', 'StandardHours'] 
for x in col_drop:
    df = df.drop(x, axis=1)


In [34]:
df_1 = df.loc[df.Attrition == 1]
df_0 = df.loc[df.Attrition == 0]


In [35]:
categorical_col = list(set(df.columns) - set(numerical_col))
df_1_sample = df_1.sample(replace=False,
                        n=200,
                        random_state=42)
df_0_sample = df_0.sample(replace=False,
                        n=200,
                        random_state=42)


In [36]:
df_sample = pd.concat((df_1_sample,df_0_sample ))
cols = df_sample.columns
X = df_sample.drop(cols[1], axis=1)
y = df_sample['Attrition']

In [37]:
num_col = X.columns
X_encode = pd.get_dummies(X, columns=categorical_col)


In [47]:
pd.set_option('display.max_columns', 500)

In [60]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_encode.values, y, test_size=100, random_state=0)


In [63]:
from sklearn.preprocessing import StandardScaler
i = 22
print(X_train[:, :i+1])
scaler = StandardScaler().fit(X_train[:, :i+1])


[[49 636 10 ... 2 1 2]
 [31 359 18 ... 0 0 0]
 [33 970 7 ... 2 0 2]
 ...
 [39 360 23 ... 2 0 3]
 [51 1323 4 ... 0 2 7]
 [33 118 16 ... 2 0 2]]


array([[49, 636, 10, ..., False, True, False],
       [31, 359, 18, ..., True, False, False],
       [33, 970, 7, ..., False, False, True],
       ...,
       [39, 360, 23, ..., False, True, False],
       [51, 1323, 4, ..., False, True, False],
       [33, 118, 16, ..., False, False, True]], dtype=object)

In [74]:

def preproces(x):
    A = np.copy(x)
    A[:,:i+1] = scaler.transform(x[:, :i+1]) 
    return A
preproces(X_train)

-9.71445146547012e-17

In [76]:
from sklearn.preprocessing import FunctionTransformer
preproces_transformer = FunctionTransformer(preproces)

In [79]:
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
pipe1 = Pipeline([("Scaler", preproces_transformer), ("clf1",DecisionTreeClassifier(random_state=0) )])
pipe2 = Pipeline([("Scaler", preproces_transformer), ("clf2",SVC(random_state=0) )])
pipe3 = Pipeline([("Scaler", preproces_transformer), ("clf3",LogisticRegression(random_state=0) )])

In [83]:
p1 = {
    "clf1__max_depth" : (2,3,5,7,10,15),
    "clf1__min_samples_leaf" :(3,5,7,9,15,20),
    "clf1__min_samples_split" : (3,5,8,10)
}
from sklearn.model_selection import GridSearchCV
grid1 = GridSearchCV(pipe1, p1, cv=5, scoring="accuracy", verbose=4)

In [84]:
grid1.fit(X_train, y_train)

Fitting 5 folds for each of 144 candidates, totalling 720 fits
[CV 1/5] END clf1__max_depth=2, clf1__min_samples_leaf=3, clf1__min_samples_split=3;, score=0.517 total time=   0.0s
[CV 2/5] END clf1__max_depth=2, clf1__min_samples_leaf=3, clf1__min_samples_split=3;, score=0.567 total time=   0.0s
[CV 3/5] END clf1__max_depth=2, clf1__min_samples_leaf=3, clf1__min_samples_split=3;, score=0.700 total time=   0.0s
[CV 4/5] END clf1__max_depth=2, clf1__min_samples_leaf=3, clf1__min_samples_split=3;, score=0.550 total time=   0.0s
[CV 5/5] END clf1__max_depth=2, clf1__min_samples_leaf=3, clf1__min_samples_split=3;, score=0.633 total time=   0.0s
[CV 1/5] END clf1__max_depth=2, clf1__min_samples_leaf=3, clf1__min_samples_split=5;, score=0.517 total time=   0.0s
[CV 2/5] END clf1__max_depth=2, clf1__min_samples_leaf=3, clf1__min_samples_split=5;, score=0.567 total time=   0.0s
[CV 3/5] END clf1__max_depth=2, clf1__min_samples_leaf=3, clf1__min_samples_split=5;, score=0.700 total time=   0.0s
[

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('Scaler',
                                        FunctionTransformer(func=<function preproces at 0x0000019F89414E50>)),
                                       ('clf1',
                                        DecisionTreeClassifier(random_state=0))]),
             param_grid={'clf1__max_depth': (2, 3, 5, 7, 10, 15),
                         'clf1__min_samples_leaf': (3, 5, 7, 9, 15, 20),
                         'clf1__min_samples_split': (3, 5, 8, 10)},
             scoring='accuracy', verbose=4)

In [87]:
grid1.best_score_
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
print(classification_report(y_test, grid1.predict(X_test)))
print(confusion_matrix(y_test, grid1.predict(X_test)))


              precision    recall  f1-score   support

           0       0.62      0.76      0.68        45
           1       0.76      0.62      0.68        55

    accuracy                           0.68       100
   macro avg       0.69      0.69      0.68       100
weighted avg       0.69      0.68      0.68       100

[[34 11]
 [21 34]]


In [90]:
p2 = {"clf2__C" : (0.5, 1, 10, 100),
     "clf2__gamma" : ("scale", 1, 0.1, 0.01, 0.001, 0.0001),
     "clf2__kernel" :["rbf"]
     }
grid2 = GridSearchCV(pipe2, p2, cv=5, scoring="accuracy", verbose=4)

In [91]:
grid2.fit(X_train, y_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
[CV 1/5] END clf2__C=0.5, clf2__gamma=scale, clf2__kernel=rbf;, score=0.717 total time=   0.0s
[CV 2/5] END clf2__C=0.5, clf2__gamma=scale, clf2__kernel=rbf;, score=0.683 total time=   0.0s
[CV 3/5] END clf2__C=0.5, clf2__gamma=scale, clf2__kernel=rbf;, score=0.750 total time=   0.0s
[CV 4/5] END clf2__C=0.5, clf2__gamma=scale, clf2__kernel=rbf;, score=0.717 total time=   0.0s
[CV 5/5] END clf2__C=0.5, clf2__gamma=scale, clf2__kernel=rbf;, score=0.700 total time=   0.0s
[CV 1/5] END clf2__C=0.5, clf2__gamma=1, clf2__kernel=rbf;, score=0.517 total time=   0.0s
[CV 2/5] END clf2__C=0.5, clf2__gamma=1, clf2__kernel=rbf;, score=0.517 total time=   0.0s
[CV 3/5] END clf2__C=0.5, clf2__gamma=1, clf2__kernel=rbf;, score=0.517 total time=   0.0s
[CV 4/5] END clf2__C=0.5, clf2__gamma=1, clf2__kernel=rbf;, score=0.517 total time=   0.0s
[CV 5/5] END clf2__C=0.5, clf2__gamma=1, clf2__kernel=rbf;, score=0.517 total time=   0.0s
[CV 1/5]

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('Scaler',
                                        FunctionTransformer(func=<function preproces at 0x0000019F89414E50>)),
                                       ('clf2', SVC(random_state=0))]),
             param_grid={'clf2__C': (0.5, 1, 10, 100),
                         'clf2__gamma': ('scale', 1, 0.1, 0.01, 0.001, 0.0001),
                         'clf2__kernel': ['rbf']},
             scoring='accuracy', verbose=4)

In [100]:
print(classification_report(y_test, grid2.predict(X_test)))
print(confusion_matrix(y_test, grid2.predict(X_test)))
print(grid2.best_params_)

              precision    recall  f1-score   support

           0       0.75      0.91      0.82        45
           1       0.91      0.75      0.82        55

    accuracy                           0.82       100
   macro avg       0.83      0.83      0.82       100
weighted avg       0.84      0.82      0.82       100

[[41  4]
 [14 41]]
{'clf2__C': 100, 'clf2__gamma': 0.001, 'clf2__kernel': 'rbf'}


In [96]:
p3 = {
    "clf3__C" : (10, 5, 1, 0.1, 0.01, 0.001, 0.0001)
    }
grid3 = GridSearchCV(pipe3, p3, cv=5, scoring="accuracy", verbose=4)

In [97]:
grid3.fit(X_train, y_train)

Fitting 5 folds for each of 7 candidates, totalling 35 fits
[CV 1/5] END ........................clf3__C=10;, score=0.733 total time=   0.0s
[CV 2/5] END ........................clf3__C=10;, score=0.617 total time=   0.0s


C:\Users\informatika\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\informatika\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-le

[CV 3/5] END ........................clf3__C=10;, score=0.800 total time=   0.0s
[CV 4/5] END ........................clf3__C=10;, score=0.733 total time=   0.0s
[CV 5/5] END ........................clf3__C=10;, score=0.750 total time=   0.0s
[CV 1/5] END .........................clf3__C=5;, score=0.750 total time=   0.0s
[CV 2/5] END .........................clf3__C=5;, score=0.617 total time=   0.0s


C:\Users\informatika\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\informatika\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-le

[CV 3/5] END .........................clf3__C=5;, score=0.800 total time=   0.0s
[CV 4/5] END .........................clf3__C=5;, score=0.733 total time=   0.0s
[CV 5/5] END .........................clf3__C=5;, score=0.750 total time=   0.0s
[CV 1/5] END .........................clf3__C=1;, score=0.683 total time=   0.0s
[CV 2/5] END .........................clf3__C=1;, score=0.650 total time=   0.0s


C:\Users\informatika\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV 3/5] END .........................clf3__C=1;, score=0.800 total time=   0.0s
[CV 4/5] END .........................clf3__C=1;, score=0.750 total time=   0.0s
[CV 5/5] END .........................clf3__C=1;, score=0.733 total time=   0.0s
[CV 1/5] END .......................clf3__C=0.1;, score=0.733 total time=   0.0s
[CV 2/5] END .......................clf3__C=0.1;, score=0.683 total time=   0.0s
[CV 3/5] END .......................clf3__C=0.1;, score=0.783 total time=   0.0s
[CV 4/5] END .......................clf3__C=0.1;, score=0.700 total time=   0.0s
[CV 5/5] END .......................clf3__C=0.1;, score=0.717 total time=   0.0s
[CV 1/5] END ......................clf3__C=0.01;, score=0.717 total time=   0.0s
[CV 2/5] END ......................clf3__C=0.01;, score=0.650 total time=   0.0s
[CV 3/5] END ......................clf3__C=0.01;, score=0.733 total time=   0.0s
[CV 4/5] END ......................clf3__C=0.01;, score=0.733 total time=   0.0s
[CV 5/5] END ...............

C:\Users\informatika\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('Scaler',
                                        FunctionTransformer(func=<function preproces at 0x0000019F89414E50>)),
                                       ('clf3',
                                        LogisticRegression(random_state=0))]),
             param_grid={'clf3__C': (10, 5, 1, 0.1, 0.01, 0.001, 0.0001)},
             scoring='accuracy', verbose=4)

In [99]:
grid3.best_score_
print(classification_report(y_test, grid3.predict(X_test)))

              precision    recall  f1-score   support

           0       0.70      0.84      0.77        45
           1       0.85      0.71      0.77        55

    accuracy                           0.77       100
   macro avg       0.78      0.78      0.77       100
weighted avg       0.78      0.77      0.77       100

